In [9]:
# read dataset one image
from keras.preprocessing import image

imageSize = 128
is_debug = False

def readImage(path):
    img = image.load_img(path, target_size=(imageSize, imageSize))
    x = image.img_to_array(img)
    x = x.astype('float32')
    x /= 255
    return x

In [10]:
def getLabel(path):
    name = path[path.rfind('/') + 1:path.index('.')]
#     print (name)
    return 1 if name == 'dog' else 0
print ("getLabel done")

getLabel done


In [11]:
import helper

def readFeatureAndLabel():
    train_files = helper.get_train_files()
    file_count = 1024 if is_debug else len(train_files)
    print("readFeatureAndLabel file_count=" + str(file_count))
    train_feature = []
    train_label = []
    for i in range(file_count):
        f = train_files[i]
#         print (f)
        train_feature.append(readImage(f))
        train_label.append(getLabel(f))
    
    return train_feature, train_label

In [12]:
import numpy as np

total_feature, total_label = readFeatureAndLabel()
total_feature = np.array(total_feature)
# total_feature = total_feature.astype('float32')
# total_feature /= 255

total_label = np.array(total_label)
# print (total_label)

readFeatureAndLabel file_count=1024


In [13]:
# split train_feature, train_label, validation_feature, validation_label
from sklearn.model_selection import train_test_split
train_feature, validation_feature, train_label, validation_label = train_test_split(total_feature, total_label, test_size=0.1, random_state=1)
    
# print (train_label)

In [20]:
print(type(train_feature))

<class 'numpy.ndarray'>


In [14]:
import math

def score_fun(label_pre):
    total_score = 0
    size = validation_label.size
    for i in range(size):
        y = validation_label[i]
        yp = label_pre[i]
#         print (str(y) + " " + str(yp))
        try:
            total_score = y * math.log(yp) + (1 - y)*math.log(1-yp)
        except:
            print (str(i) + ": " + str(y) + " - " + str(yp))
    return -total_score/size

In [15]:
# convolutional
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(imageSize, imageSize, 3)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(train_feature, train_label, batch_size=16, epochs=10, validation_data=(validation_feature, validation_label))

pre_val_label = model.predict(x=validation_feature, batch_size=16)
# for i in range(pre_val_label.size):
#     print(str(i) + ": " + str(pre_val_label[i]))
score = model.evaluate(validation_feature, validation_label, batch_size=16)
print (score)


Train on 921 samples, validate on 103 samples
Epoch 1/10
921/921 [==============================] - 92s 100ms/step - loss: 0.8263 - acc: 0.5136 - val_loss: 0.6927 - val_acc: 0.5243
Epoch 2/10
921/921 [==============================] - 93s 101ms/step - loss: 0.7008 - acc: 0.5472 - val_loss: 0.7358 - val_acc: 0.4757
Epoch 3/10
921/921 [==============================] - 93s 101ms/step - loss: 0.6739 - acc: 0.6135 - val_loss: 0.6864 - val_acc: 0.5728
Epoch 4/10
921/921 [==============================] - 99s 108ms/step - loss: 0.6825 - acc: 0.6352 - val_loss: 0.7054 - val_acc: 0.5340
Epoch 5/10
921/921 [==============================] - 99s 108ms/step - loss: 0.6277 - acc: 0.6710 - val_loss: 1.1850 - val_acc: 0.4951
Epoch 6/10
921/921 [==============================] - 102s 111ms/step - loss: 0.5932 - acc: 0.7177 - val_loss: 0.7396 - val_acc: 0.6214
Epoch 7/10
921/921 [==============================] - 94s 102ms/step - loss: 0.4851 - acc: 0.7709 - val_loss: 0.7489 - val_acc: 0.5631
Epoch 8/

In [16]:
# score the model

score = score_fun(pre_val_label)

print (score)

0.0013107780161161018


In [17]:
# save weights
model.save_weights('weights.h5')

In [23]:
import helper

test_files = helper.get_test_files()
test_files = sorted(test_files, key=lambda s: (len(s), s))

test_file_size = 128 if is_debug else len(test_files)

test_feature = []
for i in range(test_file_size):
    test_feature.append(readImage(test_files[i]))
test_feature = np.asarray(test_feature)
print(type(test_feature))
test_label = model.predict(test_feature, batch_size=16)
print (len(test_label))

<class 'numpy.ndarray'>
128


In [37]:
test_label_output = []
for i in range(len(test_label)):
    test_label_output.append([i + 1, test_label[i]])
    
print (len(test_label_output))
test_label_output = np.array(test_label_output)
np.savetxt("test.csv", test_label_output, delimiter=',', header="id,label", comments="")
# np.savetxt('test.csv', test_label, delimiter=",")

128
